In [2]:
import numpy as np
import pandas as pd
import time
from scipy.spatial import distance
from scipy import stats
import operator

Why does sklearn have a Knn fit() function?
- https://stats.stackexchange.com/questions/349842/why-do-we-need-to-fit-a-k-nearest-neighbors-classifier

In [3]:
train = pd.read_csv('data/MNIST_train_small.csv', header=None, names=['label',*np.arange(start=1,stop=785,step=1)])
test = pd.read_csv('data/MNIST_test_small.csv', header=None, names=['label',*np.arange(start=1,stop=785,step=1)])

In [4]:
train_big = pd.read_csv('data/MNIST_train.csv', header=None, names=['label',*np.arange(start=1,stop=785,step=1)])

FileNotFoundError: [Errno 2] File b'data/MNIST_train.csv' does not exist: b'data/MNIST_train.csv'

In [ ]:
train.head()

Algorithm for K-NN:
1.   Initialize k
2.   For each row:
    1. Calculate distance between the data point and each data points in the given data file.
    2. Then add the distances corresponding to rows in given data file (probably by adding column for distance).
    3. Sort the data in data file from smallest to largest (in ascending order) by the distances.
3.   Pick the first K entries from the sorted collection of data.
4.   Observe the labels of the selected K entries.
5.   For classification, return the mode of the K labels and for regression, return the mean of K labels.

In [5]:
import time
import pickle
from tqdm import tqdm,tqdm_notebook


def minkowski_dist(v,w,p=2):
    D = np.abs(np.subtract(w,v))
    Dp = np.power(D,p)
    distance_vals = np.sum(Dp, axis=0)
    distance_vals_p = np.power(distance_vals,1/p)
    return distance_vals_p


def minkowski_distances(v,w,p_list=[1,2,3]):
    D = np.abs(np.subtract(w,v))
    dist_list = []
    for p in p_list:
        Dp = np.power(D,p)
        distance_vals = np.sum(Dp, axis=0)
        distance_vals_p = np.power(distance_vals,1/p)
        dist_list.append(distance_vals_p)
    return dist_list



def create_distance_dict (dataset, p_list=[1,2,3], max_k=100, filename="dist.pkl"):
    distance_dict = {}
    for p in p_list:
        distance_dict[str(p)] = {}

    for image_inx, image_row in tqdm_notebook(dataset.iterrows(), total=dataset.shape[0]):

        image_label = image_row['label']
        image_image = image_row[1:]

        image_list_matrix = np.asarray(train.iloc[:,1:]).T
        image_single_matrix = np.asarray(image_image).reshape(784,1)

        distances = minkowski_distances(image_list_matrix,image_single_matrix, p_list)
        for ind in range(len(distances)):
            dist = distances[ind]
            p = p_list[ind]

            dist_label = np.c_[dist, dataset['label']]  # append the training labels: [distance, label]
            dist_label_sort = dist_label[dist_label[:,0].argsort(kind='quicksort')]  # sort the distances

            first_k_labels = dist_label_sort[:max_k, 1]

            distance_dict[str(p)][str(image_inx)] = first_k_labels
            pickle.dump(distance_dict, open(filename, "wb"))
        


dataset = train
p_list= [k+1 for k in range(3)]

create_distance_dict (train, p_list=p_list, max_k=100, filename="dist_tillp15.pkl")    


# print(distance_dict)
#distance_df.to_pickle("./euclidean_small.pkl")

"""
v = np.random.rand(784,1)
w = np.random.rand(784,50000)



from sklearn.metrics import pairwise_distances

pairwise_distances(v.T,w.T, metric='euclidean')
second_time = time.time()

minkowski_dist(v,w)
third_time = time.time()

elapsed_time = time.time() - start    
print(second_time-start)
print(third_time-second_time)  
"""

KeyboardInterrupt: 

In [9]:
dist_df = pickle.load(open("dist.pkl", "rb"))

def get_knn_labels (img_index, minkowski_p=2, k=20):
    nn_labels = dist_df[str(minkowski_p)][str(img_index)][:k]
    return nn_labels

get_knn_labels(img_index=1945, minkowski_p=2, k=20)

array([4., 4., 4., 4., 7., 7., 9., 4., 4., 4., 7., 4., 4., 4., 9., 9., 7.,
       4., 4., 4.])

In [ ]:
# euclidean distance
def distance_euclid(a_vector, b_vector):
    pass

# more distance functions?


# loss function
def loss_01(predictions, labels):
    losses = np.equal(predictions, labels)
    return losses
    
    
def get_emp_risk(k_losses):
    emp_risks = np.matrix(np.zeros((1, k_losses.shape[1])))

    for col in range(k_losses.shape[1]):
        n = k_losses.shape[0]  # number of image images
        sum_loss = k_losses.sum(axis=0)[0,col]  # sum up losses for all image images for one k value
        emp_risks[0,col] = sum_loss/n  # the average loss
    return emp_risks  


# classify each image in image data using each image in training data
def knn(dataset, k_list, distance_metric):

    # data structures for accumulating predictions and losses over all k values
    k_predictions = np.matrix(np.zeros((dataset.shape[0], len(k_list))))  # matrix of predictions for each k value. 
    k_losses = np.matrix(np.zeros((dataset.shape[0], len(k_list))))  # matrix of losses for each k value. 

    ''' Classify one image image at a time using all training data '''
    for image_inx, image_row in dataset.iterrows():

        image_label = image_row['label']
        image_image = image_row[1:]

        dist = distance_metric(dataset.iloc[:,1:], [image_image])  # distance matrix operation
        dist_label = np.c_[dist, dataset['label']]  # append the training labels: [distance, label]
        dist_label_sort = dist_label[dist_label[:,0].argsort(kind='quicksort')]  # sort the distances

        ''' Determine prediction and loss for the image image using each k value '''
        for i in range(len(k_list)):
            
            first_k_labels = dist_label_sort[:k_list[i], 1]  # labels from the closest k training images
            prediction = stats.mode(first_k_labels)[0][0]  # the mode of the labels is the prediction
            loss = loss_01(prediction, image_row['label'])  # determine the 0-1 loss
            
            k_predictions[image_inx,i] = prediction  # store the prediction for the image image with this k
            k_losses[image_inx,i] = loss  # store the loss for the image image with this k

    return get_emp_risk(k_losses) # compute the emperical risk over all classified image images for each k-value


# Determine which distance metric is the fasimage
def grid_search(dataset, k_list, distance_list): 
    
    results_list = []

    for distance_inx in range(len(distance_list)):

        start = time.time()  # begin timer
        emp_risks = knn(dataset, k_list, distance_list[distance_inx])
        elapsed_time = time.time() - start # end timer

        results_list.append((elapsed_time, emp_risks))

    return results_list  # (computation time, emperical risk)

In [20]:
# works but takes ~1 sec per test image. Look into ways to speed up distance calculations.
# about 12 seconds to run

grid_search(train_big, [2,4], [distance.euclidean])

ValueError: Input vector should be 1-D.

In [ ]:
# report on its accuracy to predict the labels in both the training and test sets
# plot your the empirical training and test loss as a function of k

d) Building up on your work for the previous questions
suggest a different distance metric or some pre-processing of the data that you consider
appropriate to improve the performance of the k-NN method. Note that, any choices
you make should be done solely based on the training data (that is, do not clairvoyantly
optimize the performance of your method on the test data). Clearly justify ALL the
choices made and describe the exact steps you took. Someone reading your report
should be able to replicate your results.
- Normalize the data for the distance calculations